In [1]:
import pandas as pd
import pickle
import argparse
import os
import glob
import pydicom
import sys

/tmp/ipykernel_2777912/1002562574.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
combined_roi_dict = {
    'Left-Cingulate': {
        'ctx-lh-caudalanteriorcingulate',
        'ctx-lh-rostralanteriorcingulate',
        'ctx-lh-isthmuscingulate',
        'ctx-lh-posteriorcingulate',
    },
    'Right-Cingulate': {
        'ctx-rh-caudalanteriorcingulate',
        'ctx-rh-rostralanteriorcingulate',
        'ctx-rh-isthmuscingulate',
        'ctx-rh-posteriorcingulate',
    },
    'Left-Frontal': {
        'ctx-lh-caudalmiddlefrontal',
        'ctx-lh-lateralorbitofrontal',
        'ctx-lh-medialorbitofrontal',
        'ctx-lh-parsopercularis',
        'ctx-lh-parsorbitalis',
        'ctx-lh-parstriangularis',
        'ctx-lh-precentral',
        'ctx-lh-rostralmiddlefrontal',
        'ctx-lh-superiorfrontal',
        'ctx-lh-paracentral'
    },
    'Right-Frontal': {
        'ctx-rh-caudalmiddlefrontal',
        'ctx-rh-lateralorbitofrontal',
        'ctx-rh-medialorbitofrontal',
        'ctx-rh-parsopercularis',
        'ctx-rh-parsorbitalis',
        'ctx-rh-parstriangularis',
        'ctx-rh-precentral',
        'ctx-rh-rostralmiddlefrontal',
        'ctx-rh-superiorfrontal',
        'ctx-rh-paracentral'
    },
    'Left-Temporal': {
        'ctx-lh-entorhinal',
        'ctx-lh-fusiform',
        'ctx-lh-inferiortemporal',
        'ctx-lh-middletemporal',
        'ctx-lh-superiortemporal',
        'ctx-lh-transversetemporal',
        'ctx-lh-parahippocampal',
        'Left-Hippocampus',
        'Left-Amygdala',
    },
    'Right-Temporal': {
        'ctx-rh-entorhinal',
        'ctx-rh-fusiform',
        'ctx-rh-inferiortemporal',
        'ctx-rh-middletemporal',
        'ctx-rh-superiortemporal',
        'ctx-rh-transversetemporal',
        'ctx-rh-parahippocampal',
        'Right-Hippocampus',
        'Right-Amygdala',
    },
    'Left-Parietal': {
        'ctx-lh-inferiorparietal',
        'ctx-lh-postcentral',
        'ctx-lh-superiorparietal',
        'ctx-lh-supramarginal',
        'ctx-lh-precuneus'
    },
    'Right-Parietal': {
        'ctx-rh-inferiorparietal',
        'ctx-rh-postcentral',
        'ctx-rh-superiorparietal',
        'ctx-rh-supramarginal',
        'ctx-rh-precuneus'
    },
    'Left-Occipital': {
        'ctx-lh-cuneus',
        'ctx-lh-lateraloccipital',
        'ctx-lh-lingual',
        'ctx-lh-pericalcarine'
    },
    'Right-Occipital': {
        'ctx-rh-cuneus',
        'ctx-rh-lateraloccipital',
        'ctx-rh-lingual',
        'ctx-rh-pericalcarine'
    },
    'Left-Insula':{
        'ctx-lh-insula'
    },
    'Right-Insula': {
        'ctx-rh-insula'
    },
    'Left-Cerebellum':{
        'Left-Cerebellum-White-Matter',
        'Left-Cerebellum-Cortex'
    },
    'Right-Cerebellum': {
        'Right-Cerebellum-White-Matter',
        'Right-Cerebellum-Cortex'
    },
    'Left-Combined-Cerebral-WM': {
        'Left-Cerebral-White-Matter',
        'Left-WM-hypointensities'
    },
    'Right-Combined-Cerebral-WM': {
        'Right-Cerebral-White-Matter',
        'Right-WM-hypointensities'
    },
    'Left-Total-Lateral-Ventricle': {
        'Left-Lateral-Ventricle',
        'Left-Inf-Lat-Vent'
    },
    'Right-Total-Lateral-Ventricle': {
        'Right-Lateral-Ventricle',
        'Right-Inf-Lat-Vent'
    },
}
gray_matter_index = {
    'left_cortical_gm': {
        'left_cingulate',
        'left_frontal',
        'left_temporal',
        'left_parietal',
        'left_occipital',
        'left_insula'
    },
    'right_cortical_gm': {
        'right_cingulate',
        'right_frontal',
        'right_temporal',
        'right_parietal',
        'right_occipital',
        'right_insula'
    }
}

combined_roi_display_dict = {
    'Left-Cingulate': {
        'ctx-lh-caudalanteriorcingulate',
        'ctx-lh-rostralanteriorcingulate',
        'ctx-lh-isthmuscingulate',
        'ctx-lh-posteriorcingulate',
    },
    'Right-Cingulate': {
        'ctx-rh-caudalanteriorcingulate',
        'ctx-rh-rostralanteriorcingulate',
        'ctx-rh-isthmuscingulate',
        'ctx-rh-posteriorcingulate',
    },
    'Left-Frontal': {
        'ctx-lh-caudalmiddlefrontal',
        'ctx-lh-lateralorbitofrontal',
        'ctx-lh-medialorbitofrontal',
        'ctx-lh-parsopercularis',
        'ctx-lh-parsorbitalis',
        'ctx-lh-parstriangularis',
        'ctx-lh-precentral',
        'ctx-lh-rostralmiddlefrontal',
        'ctx-lh-superiorfrontal',
        'ctx-lh-paracentral'
    },
    'Right-Frontal': {
        'ctx-rh-caudalmiddlefrontal',
        'ctx-rh-lateralorbitofrontal',
        'ctx-rh-medialorbitofrontal',
        'ctx-rh-parsopercularis',
        'ctx-rh-parsorbitalis',
        'ctx-rh-parstriangularis',
        'ctx-rh-precentral',
        'ctx-rh-rostralmiddlefrontal',
        'ctx-rh-superiorfrontal',
        'ctx-rh-paracentral'
    },
    'Left-Temporal': {
        'ctx-lh-entorhinal',
        'ctx-lh-fusiform',
        'ctx-lh-inferiortemporal',
        'ctx-lh-middletemporal',
        'ctx-lh-superiortemporal',
        'ctx-lh-transversetemporal',
        'ctx-lh-parahippocampal',
    },
    'Right-Temporal': {
        'ctx-rh-entorhinal',
        'ctx-rh-fusiform',
        'ctx-rh-inferiortemporal',
        'ctx-rh-middletemporal',
        'ctx-rh-superiortemporal',
        'ctx-rh-transversetemporal',
        'ctx-rh-parahippocampal',
    },
    'Left-Parietal': {
        'ctx-lh-inferiorparietal',
        'ctx-lh-postcentral',
        'ctx-lh-superiorparietal',
        'ctx-lh-supramarginal',
        'ctx-lh-precuneus'
    },
    'Right-Parietal': {
        'ctx-rh-inferiorparietal',
        'ctx-rh-postcentral',
        'ctx-rh-superiorparietal',
        'ctx-rh-supramarginal',
        'ctx-rh-precuneus'
    },
    'Left-Occipital': {
        'ctx-lh-cuneus',
        'ctx-lh-lateraloccipital',
        'ctx-lh-lingual',
        'ctx-lh-pericalcarine'
    },
    'Right-Occipital': {
        'ctx-rh-cuneus',
        'ctx-rh-lateraloccipital',
        'ctx-rh-lingual',
        'ctx-rh-pericalcarine'
    },
    'Left-Insula':{
        'ctx-lh-insula'
    },
    'Right-Insula': {
        'ctx-rh-insula'
    },
    'Left-Cerebellum':{
        'Left-Cerebellum-White-Matter',
        'Left-Cerebellum-Cortex'
    },
    'Right-Cerebellum': {
        'Right-Cerebellum-White-Matter',
        'Right-Cerebellum-Cortex'
    },
    'Left-Combined-Cerebral-WM': {
        'Left-Cerebral-White-Matter',
        'Left-WM-hypointensities'
    },
    'Right-Combined-Cerebral-WM': {
        'Right-Cerebral-White-Matter',
        'Right-WM-hypointensities'
    }
}

In [3]:
def load_pickle_file(filepath):
    with open(filepath, 'rb') as file:
        data = pickle.load(file)
    return data
# 파일열어보기 , PICKLE로 되어있는것

In [4]:
import os
import pandas as pd
import pickle

def load_pickle_file(file_path):
    with open(file_path, 'rb') as file:
        return pickle.load(file)
        
def find_volumes_files(root_path):
    volumes_files = []
    for root, dirs, files in os.walk(root_path):
        for file in files:
            if file == 'volumes.pickle':
                volumes_files.append(os.path.join(root, file))
    return volumes_files

def extract_subject_data(volumes_file_path):
    subject_name = os.path.dirname(volumes_file_path).split(os.sep)[-3]
    volumes_data = load_pickle_file(volumes_file_path)
    volumes_data['subject'] = subject_name
    return volumes_data

def compile_data(volumes_files):
    data = []
    for volumes_file in volumes_files:
        subject_data = extract_subject_data(volumes_file)
        data.append(subject_data)
    df = pd.DataFrame(data)
    # You might need to adjust the DataFrame to ensure it has the structure you want
    return df

root_path = '/home/limseoyoung/drwelly1/SeoyoungLim/drwelly-debug/drwelly_oasis2_after_swift'
volumes_files = find_volumes_files(root_path)
df = compile_data(volumes_files)


In [30]:
df

,total_intracranial_volume,unknown,left_cerebral_white_matter,left_lateral_ventricle,left_inf_lat_vent,left_cerebellum_white_matter,left_cerebellum_cortex,left_thalamus,left_caudate,left_putamen,...,cerebellum_white_matter,cerebellum_cortex,lateral_ventricle,inf_lat_vent,total_lateral_ventricle,wm_hypointensities,HippocampusOccupancyScore,HippocampusAsymmetryIndex,subject,sort_key
0,1.450921e+06,1.536565e+07,193320.621123,25387.829703,1758.217910,10182.983480,40608.453925,6381.857674,3190.585837,4445.622773,...,19909.090983,82016.440810,49482.892472,3434.458825,52917.351296,5997.354694,0.591583,0.037629,OAS2_0063_MR2_mpr-1,63
1,1.424640e+06,1.539193e+07,194662.780055,8990.372541,853.726956,10540.732461,46476.840626,6981.764643,2798.879857,4998.195745,...,20698.333943,92210.057289,17186.357122,1667.665915,18854.023037,908.949954,0.799059,0.019283,OAS2_0174_MR3_mpr-2,174
2,1.461654e+06,1.535491e+07,205391.819507,15919.658187,1259.152936,11671.946404,50986.947395,6180.173684,3645.403814,4356.099777,...,22910.683830,103274.550724,31402.334396,2636.297865,34038.632261,2274.089884,0.698992,0.001570,OAS2_0056_MR2_mpr-2,56
3,1.630912e+06,1.518566e+07,199354.676815,30112.654462,2232.243886,13086.478331,54778.812201,7178.646633,4080.327792,4830.811753,...,26266.252658,110116.028374,60547.042907,4253.199783,64800.242689,21827.489885,0.636985,0.019586,OAS2_0005_MR1_mpr-3,5
4,1.498738e+06,1.531783e+07,220045.464758,33350.231296,1951.669900,11057.290435,45448.526678,6747.152655,3152.169839,4036.423794,...,21569.553898,91229.763339,64287.800716,3523.981820,67811.782536,25572.020694,0.590019,0.082173,OAS2_0053_MR2_mpr-2,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1356,1.516163e+06,1.530040e+07,194466.927065,24813.647732,1618.959917,10309.550473,46084.105646,5508.236719,3137.763840,4363.988777,...,19998.270978,91526.458324,45716.409664,2843.126855,48559.536519,6217.560682,0.704002,0.023732,OAS2_0080_MR1_mpr-1,80
1357,1.528861e+06,1.528771e+07,203518.010602,25607.349692,1598.379918,11719.966401,46007.273650,6975.933644,4247.711783,4861.338752,...,22788.232836,92053.649297,50154.486438,3027.660845,53182.147283,18441.051058,0.669871,0.018489,OAS2_0178_MR3_mpr-3,178
1358,1.448261e+06,1.536831e+07,202321.969663,31451.040393,2018.554897,16490.753157,50504.003420,6936.488646,3782.603807,4194.203786,...,32233.080353,100063.727888,57075.540084,4322.828779,61398.368863,7318.933626,0.626861,0.069789,OAS2_0046_MR1_mpr-3,46
1359,1.388052e+06,1.542852e+07,189355.541326,14648.843252,994.356949,10849.089446,40483.601932,6181.888684,2884.286853,4079.984792,...,21299.612912,82820.089769,28588.705539,1921.142902,30509.848441,2208.576887,0.780467,0.022035,OAS2_0027_MR3_mpr-1,27


In [31]:
df['subject'] = df['subject'].str.replace('_Tag_pr100_sr100_ppf100_spf100_ipf100_sipf179', '')


In [7]:
# 'mpr-x'에 따라 4개의 데이터프레임으로 분할
#volume_df_mpr1 = df[df['subject'].str.contains('MR1_mpr-1')]
#volume_df_mpr2 = df[df['subject'].str.contains('MR1_mpr-2')]
#volume_df_mpr3 = df[df['subject'].str.contains('MR1_mpr-3')]
#volume_df_mpr4 = df[df['subject'].str.contains('MR1_mpr-4')]

In [32]:
# 'mpr-x'에 따라 4개의 데이터프레임으로 분할
volume_df_mpr1 = df[df['subject'].str.contains('MR1')]
volume_df_mpr2 = df[df['subject'].str.contains('MR2')]
volume_df_mpr3 = df[df['subject'].str.contains('MR3')]
volume_df_mpr4 = df[df['subject'].str.contains('MR4')]

In [33]:
volume_df_mpr1.shape

(582, 145)

In [34]:
df.shape

(1361, 145)

In [35]:

def extract_number(subj):
    parts = subj.split('_')  # '_'를 기준으로 문자열 분리
    if len(parts) > 1:
        number_part = parts[1]  # "OAS1_" 다음 오는 부분을 선택
        # 숫자만 추출
        numbers = ''.join(filter(str.isdigit, number_part))
        return int(numbers) if numbers.isdigit() else None
    return None

In [36]:

volume_df_mpr1 ['sort_key'] = volume_df_mpr1['subject'].apply(extract_number)  # 정렬 키 생성
volume_df_sorted1 = volume_df_mpr1 .sort_values(by='sort_key').drop(columns=['sort_key'])  # 정렬 및 정렬 키 컬럼 삭제


/tmp/ipykernel_2777912/3245932642.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volume_df_mpr1 ['sort_key'] = volume_df_mpr1['subject'].apply(extract_number)  # 정렬 키 생성


In [37]:

volume_df_mpr2 ['sort_key'] = volume_df_mpr2['subject'].apply(extract_number)  # 정렬 키 생성
volume_df_sorted2 = volume_df_mpr2.sort_values(by='sort_key').drop(columns=['sort_key'])  # 정렬 및 정렬 키 컬럼 삭제


/tmp/ipykernel_2777912/4065747069.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volume_df_mpr2 ['sort_key'] = volume_df_mpr2['subject'].apply(extract_number)  # 정렬 키 생성


In [38]:

volume_df_mpr3['sort_key'] = volume_df_mpr3['subject'].apply(extract_number)  # 정렬 키 생성
volume_df_sorted3 = volume_df_mpr3.sort_values(by='sort_key').drop(columns=['sort_key'])  # 정렬 및 정렬 키 컬럼 삭제


/tmp/ipykernel_2777912/593579216.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volume_df_mpr3['sort_key'] = volume_df_mpr3['subject'].apply(extract_number)  # 정렬 키 생성


In [39]:
volume_df_mpr4['sort_key'] = volume_df_mpr4['subject'].apply(extract_number)  # 정렬 키 생성
volume_df_sorted4 = volume_df_mpr4.sort_values(by='sort_key').drop(columns=['sort_key'])  # 정렬 및 정렬 키 컬럼 삭제


/tmp/ipykernel_2777912/3330656496.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volume_df_mpr4['sort_key'] = volume_df_mpr4['subject'].apply(extract_number)  # 정렬 키 생성


In [40]:

df['sort_key'] = df['subject'].apply(extract_number)  # 정렬 키 생성
df_final= df .sort_values(by='sort_key').drop(columns=['sort_key'])  # 정렬 및 정렬 키 컬럼 삭제


In [41]:
df_final

,total_intracranial_volume,unknown,left_cerebral_white_matter,left_lateral_ventricle,left_inf_lat_vent,left_cerebellum_white_matter,left_cerebellum_cortex,left_thalamus,left_caudate,left_putamen,...,cerebellum,cerebellum_white_matter,cerebellum_cortex,lateral_ventricle,inf_lat_vent,total_lateral_ventricle,wm_hypointensities,HippocampusOccupancyScore,HippocampusAsymmetryIndex,subject
677,1.878520e+06,1.493805e+07,246361.109413,49779.930457,2725.134861,13710.052300,53966.931243,7950.053594,3386.781827,5919.150698,...,137072.396997,27476.356596,109596.040401,84107.025703,5123.047738,89230.073441,13470.295312,0.566346,0.036037,OAS2_0001_MR2_mpr-2
870,1.884769e+06,1.493180e+07,247851.444337,48707.369512,2600.968867,13393.806316,53958.013243,8050.209589,3468.415823,5991.523694,...,136283.840037,27140.216613,109143.623424,81594.893831,4729.626758,86324.520590,11526.514411,0.594755,0.035868,OAS2_0001_MR1_mpr-2
1272,1.880487e+06,1.493608e+07,245915.209436,49670.856462,2752.574859,13725.144299,53976.192242,7991.213592,3310.978831,5803.559703,...,137106.353995,27269.870607,109836.483388,83920.090713,5077.771741,88997.862453,13553.301308,0.567516,0.039484,OAS2_0001_MR2_mpr-3
1133,1.882711e+06,1.493386e+07,248085.027325,48271.073534,2626.007866,13744.352298,54350.062223,8014.537591,3466.357823,6015.876693,...,137716.207964,27649.228587,110066.979377,81191.182852,4761.525757,85952.708609,11810.518397,0.590046,0.056755,OAS2_0001_MR1_mpr-1
731,1.887315e+06,1.492925e+07,247356.838363,48490.593523,2596.166867,13748.125298,54426.208219,8042.320589,3490.710822,5927.725697,...,137835.571958,27637.909588,110197.662370,81441.229839,4741.974758,86183.204597,11315.569422,0.592415,0.054202,OAS2_0001_MR1_mpr-3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,1.369487e+06,1.544708e+07,194857.947045,11578.307408,467.508976,14208.774274,59846.636942,6544.096666,3716.747810,3856.005803,...,149620.022356,28630.551537,120989.470819,21433.039905,978.578950,22411.618855,1179.919940,0.871077,0.002024,OAS2_0186_MR3_mpr-1
1034,1.360618e+06,1.545595e+07,197342.638918,10991.777438,443.155977,13809.865294,59446.355963,6598.290663,3609.388816,3828.908804,...,147734.551452,27757.273582,119977.277870,20268.897964,987.839950,21256.737914,952.167951,0.869763,0.021407,OAS2_0186_MR2_mpr-2
479,1.367624e+06,1.544894e+07,194124.613082,11719.280401,487.745975,14297.954270,59804.104945,6532.434666,3718.805810,3848.459803,...,149573.031358,28819.887528,120753.143831,21489.634902,1014.593948,22504.228850,1212.847938,0.867298,0.008948,OAS2_0186_MR3_mpr-2
995,1.360596e+06,1.545597e+07,197895.554890,10983.888439,449.672977,14162.469276,59459.389962,6632.933661,3687.592812,3804.898806,...,147954.414441,28024.470568,119929.943873,20302.168963,928.843953,21231.012915,947.365952,0.878370,0.008334,OAS2_0186_MR2_mpr-4


In [42]:
import numpy as np

In [43]:
def update_df_columns_to_roi(df, roi_dict):
    columns_to_remove = []  # 제거할 원본 컬럼들을 저장할 리스트
    new_columns_data = {}   # 새로운 컬럼의 데이터를 저장할 사전

    # 데이터프레임의 모든 컬럼을 순회
    for col in df.columns:
        matched = False
        for roi, sub_columns in roi_dict.items():
            # 하이픈을 언더스코어로 변경한 후 매칭 시도
            if any(col.endswith(sub_col.replace('-', '_')) for sub_col in sub_columns):
                matched = True
                # 새로운 컬럼 데이터에 합산 처리
                if roi not in new_columns_data:
                    new_columns_data[roi] = df[col].fillna(0)
                else:
                    new_columns_data[roi] += df[col].fillna(0)
                
                # 컬럼 제거 리스트에 추가
                if col not in columns_to_remove:
                    columns_to_remove.append(col)
                break  # 하위 요소에 매치되면 더 이상의 검사 중단
        if not matched:
            # 매치되지 않은 컬럼은 새로운 데이터에 그대로 유지
            new_columns_data[col] = df[col]

    # 새로운 데이터프레임 생성
    new_df = pd.DataFrame(new_columns_data)

    return new_df

# 함수 실행하여 데이터프레임 업데이트, 'subject' 컬럼 유지
new_df1 = update_df_columns_to_roi(volume_df_mpr1.copy(), combined_roi_display_dict)
new_df2 = update_df_columns_to_roi(volume_df_mpr2.copy(), combined_roi_display_dict)

new_df3 = update_df_columns_to_roi(volume_df_mpr3.copy(), combined_roi_display_dict)
new_df4 = update_df_columns_to_roi(volume_df_mpr4.copy(), combined_roi_display_dict)

In [44]:
columns_to_include = [
   'whole_brain',	'cortical_gm',	'combined_cerebral_wm',	'frontal','temporal','parietal'	,'occipital','insula'	,'thalamus'	,'caudate','putamen','pallidum','hippocampus','amygdala', 'accumbens_area'	, 'ventraldc' ,'cingulate' ,'cerebellum', 'cerebellum_white_matter' ,'cerebellum_cortex','lateral_ventricle', 'inf_lat_vent',	'total_lateral_ventricle', 'wm_hypointensities',	'HippocampusOccupancyScore',	
    'subject'  # 'subject' 컬럼 추가
]

filtered_df1 = volume_df_mpr1[columns_to_include].copy()
filtered_df2 = volume_df_mpr2[columns_to_include].copy()
filtered_df3 = volume_df_mpr3[columns_to_include].copy()
filtered_df4 = volume_df_mpr4[columns_to_include].copy()

In [26]:
df_all = df[columns_to_include].copy()

In [27]:
df_all

,whole_brain,cortical_gm,combined_cerebral_wm,frontal,temporal,parietal,occipital,insula,thalamus,caudate,...,cingulate,cerebellum,cerebellum_white_matter,cerebellum_cortex,lateral_ventricle,inf_lat_vent,total_lateral_ventricle,wm_hypointensities,HippocampusOccupancyScore,subject
0,9.256578e+05,374334.745875,391898.745978,136676.232017,88659.321470,82501.442785,39755.069969,9487.722515,12587.070357,6371.567674,...,17254.957118,101925.531793,19909.090983,82016.440810,49482.892472,3434.458825,52917.351296,5997.354694,0.591583,OAS2_0063_MR2_mpr-1
1,9.806239e+05,418327.237628,389249.071113,150682.636302,103863.481694,88229.542492,43762.681764,11124.518432,13548.156308,5834.429702,...,20664.376944,112908.391232,20698.333943,92210.057289,17186.357122,1667.665915,18854.023037,908.949954,0.799059,OAS2_0174_MR3_mpr-2
2,1.010803e+06,416580.681717,409754.296066,145606.922561,102541.216761,97540.620017,39695.387972,11268.921424,12119.218381,7190.651633,...,19927.612982,126185.234553,22910.683830,103274.550724,31402.334396,2636.297865,34038.632261,2274.089884,0.698992,OAS2_0056_MR2_mpr-2
3,1.032153e+06,405400.597288,421922.563444,136681.034017,100216.019880,98088.047989,37927.909062,10901.911443,14162.469276,8422.364570,...,21585.674897,136382.281032,26266.252658,110116.028374,60547.042907,4253.199783,64800.242689,21827.489885,0.636985,OAS2_0005_MR1_mpr-3
4,1.006230e+06,372753.172956,465573.770214,131021.877306,85636.119625,87226.610544,42163.615846,9465.427516,13271.012322,6534.492666,...,17239.522119,112799.317237,21569.553898,91229.763339,64287.800716,3523.981820,67811.782536,25572.020694,0.590019,OAS2_0053_MR2_mpr-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1356,9.427173e+05,367319.024234,405298.726293,129473.575385,93722.687212,78300.036000,36450.951138,10950.960441,10971.197439,7968.575593,...,18420.814059,111524.729302,19998.270978,91526.458324,45716.409664,2843.126855,48559.536519,6217.560682,0.704002,OAS2_0080_MR1_mpr-1
1357,9.836962e+05,382204.880473,421733.913454,136759.238013,84427.044687,93699.020213,38077.457055,11338.207421,13758.072297,8597.637561,...,17903.913085,114841.882133,22788.232836,92053.649297,50154.486438,3027.660845,53182.147283,18441.051058,0.669871,OAS2_0178_MR3_mpr-3
1358,9.871656e+05,372354.606977,417235.125684,132756.771217,86947.751558,83983.202709,40878.737912,10332.531472,13409.241315,7445.843620,...,17455.612108,132296.808241,32233.080353,100063.727888,57075.540084,4322.828779,61398.368863,7318.933626,0.626861,OAS2_0046_MR1_mpr-3
1359,8.964967e+05,354554.622886,382551.310456,122351.523749,83199.790749,80711.668876,42435.271832,9425.639518,12120.933381,5939.730697,...,16430.728161,104119.702681,21299.612912,82820.089769,28588.705539,1921.142902,30509.848441,2208.576887,0.780467,OAS2_0027_MR3_mpr-1


In [45]:
filtered_df1.to_csv('session1_oasis2_atropy_20240418_after.csv')

In [46]:
filtered_df2.to_csv('session2_oasis2_atropy_20240418_after.csv')

In [47]:
filtered_df3.to_csv('session3_oasis2_atropy_20240418_after.csv')

In [48]:
filtered_df4.to_csv('session4_oasis2_atropy_20240418_after.csv')

In [28]:
#df_all.to_csv('all_oasis2_after_swift_20240417.csv')

In [29]:
pwd

'/home/limseoyoung/debug/For_tabulardata_testing'